In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Libraries
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
from sklearn import neighbors
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import cross_val_score,train_test_split,GridSearchCV
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import OrdinalEncoder
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
models = [LGBMRegressor,
          XGBRegressor,
          GradientBoostingRegressor,
          RandomForestRegressor,
          DecisionTreeRegressor,
          MLPRegressor,
          KNeighborsRegressor,
          SVR]
lbe=LabelEncoder()

In [ ]:
#collecting datas from sql

df=pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")

test=pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")
test2=pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")["Id"]


In [ ]:
df.drop(columns="Id",inplace=True)
df.drop(columns="MiscFeature",inplace=True)
df.drop(columns="Condition2",inplace=True)
test.drop(columns="Id",inplace=True)
test.drop(columns="MiscFeature",inplace=True)
test.drop(columns="Condition2",inplace=True)


In [ ]:
for i in df.columns:
    df[i].fillna(df[i].mode()[0],inplace=True)
for i in test.columns:
    test[i].fillna(test[i].mode()[0],inplace=True)

In [ ]:
def outlier_threshold(data,degisken):
    Q1=data[degisken].quantile(0.01)
    Q3=data[degisken].quantile(0.99)
    Q_Inter_Range=Q3-Q1
    alt_limit=Q1-1.5*Q_Inter_Range
    ust_limit=Q3+1.5*Q_Inter_Range
    return alt_limit,ust_limit
def threshold_degisimi(data,degisken):
    alt_limit,ust_limit=outlier_threshold(data,degisken)
    data.loc[(data[degisken] < alt_limit), degisken] = alt_limit
    data.loc[(data[degisken] > ust_limit), degisken] = ust_limit
    return data

In [ ]:
def degisken_tiplerine_ayirma(data,cat_th,car_th):
   """
   Veri:data parametresi ili fonksiyona girilen verinin değişkenlerin sınıflandırılması.
   Parameters
   ----------
   data: pandas.DataFrame
   İşlem yapılacak veri seti
   cat_th:int
   categoric değişken threshold değeri
   car_th:int
   Cardinal değişkenler için threshold değeri
   Returns
   -------
    cat_deg:list
    categorik değişken listesi
    num_deg:list
    numeric değişken listesi
    car_deg:list
    categoric ama cardinal değişken listesi
   Examples
   -------
    df = dataset_yukle("breast_cancer")
    cat,num,car=degisken_tiplerine_ayirma(df,10,20)
   Notes
   -------
    cat_deg + num_deg + car_deg = toplam değişken sayısı
   """


   num_but_cat=[i for i in data.columns if data[i].dtypes !="O" and data[i].nunique() < cat_th]

   car_deg=[i for i in data.columns if data[i].dtypes == "O" and data[i].nunique() > car_th]

   num_deg=[i for i in data.columns if data[i].dtypes !="O" and i not in num_but_cat]

   cat_deg = [i for i in data.columns if data[i].dtypes == "O" and i not in car_deg]

   cat_deg = cat_deg+num_but_cat

   print(f"Dataset kolon/değişken sayısı: {data.shape[1]}")
   print(f"Dataset satır/veri sayısı: {data.shape[0]}")
   print("********************************************")
   print(f"Datasetin numeric değişken sayısı: {len(num_deg)}")
   print(f"Datasetin numeric değişkenler: {num_deg}")
   print("********************************************")
   print(f"Datasetin categoric değişken sayısı: {len(cat_deg)}")
   print(f"Datasetin categoric değişkenler: {cat_deg}")
   print("********************************************")
   print(f"Datasetin cardinal değişken sayısı: {len(car_deg)}")
   print(f"Datasetin cardinal değişkenler: {car_deg}")
   print("********************************************")

   return cat_deg,num_deg,car_deg

In [ ]:

def feature_engineering(train,test,cat_deg,target):

    for i in cat_deg:
        l=0
        while True:
            r=pd.DataFrame(train.groupby(i).mean()[target].index.tolist(),train.groupby(i).mean()[target].values.tolist())
            r=r.reset_index()
            r=pd.DataFrame(r)
            r.columns=["a","b"]
            sozluk={}
            for j in r.index:
                sozluk[r["b"][j]]=r["a"][j]/(10**4)
            train[i].replace(sozluk,inplace=True)
            test[i].replace(sozluk, inplace=True)
            l+=1
            if l==len(train[i]):
               break
    return train,test

In [ ]:
df.head()

In [ ]:
cat_deg,num_deg,car_deg=degisken_tiplerine_ayirma(df,10,20)
cat_deg=[i for i in cat_deg if i !="SalePrice"]
data,test=feature_engineering(df,test,cat_deg+car_deg,"SalePrice")

In [ ]:
for i in data.columns:
    data=threshold_degisimi(data,i)

In [ ]:
data.head()

In [ ]:
#Deleting Outliers
clf = LocalOutlierFactor(n_neighbors = 20, contamination = 0.1)
clf.fit_predict(data)
df_scores = clf.negative_outlier_factor_
print(np.sort(df_scores)[0:20])


In [ ]:
esik_deger = np.sort(df_scores)[6]
aykiri_tf = df_scores > esik_deger
data=data[df_scores>esik_deger]

In [ ]:
af=[]
af=pd.DataFrame(af)

#comparing models eachother
def model_olusturma(dff,model,af):
    X = dff.drop(columns="SalePrice")
    X=scale(X)
    y = dff["SalePrice"]
    y= np.log1p(y)

    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        y,
                                                        test_size=0.15,
                                                        random_state=42)

    Model=model().fit(X_train,y_train)
    y_pred=Model.predict(X_test)
    y_pred=np.expm1(y_pred)
    y_test=np.expm1(y_test)
    hata_score=np.sqrt(mean_squared_error(y_test,y_pred))
    af["y_test"] = y_test
    af[""+str(model)+""]=y_pred
    print(str(model),"Hata değeri :",hata_score)
    return hata_score,af,df

In [ ]:
for i in models:
    model_olusturma(data,i,af)
    result = af.astype(int)

In [ ]:
X = data.drop(columns="SalePrice")
X=scale(X)
y = data["SalePrice"]
y= np.log1p(y)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                        y,
                                                        test_size=0.15,
                                                        random_state=42)

gbm_model = GradientBoostingRegressor().fit(X_train, y_train)
gbm_tuned = GradientBoostingRegressor(learning_rate = 0.1,
                                     loss = "lad",
                                     max_depth = 3,
                                     n_estimators = 420,
                                     subsample = 0.5).fit(X_train, y_train)
result["gbm_tuned"]=result.y_test
y_test = np.expm1(y_test)
y_train = np.expm1(y_train)
#result.gbm_tuned=gbm_tuned.predict(X_test)
#print(np.sqrt(mean_squared_error(y_test, gbm_tuned.predict(X_test))),np.sqrt(mean_squared_log_error( y_test, gbm_tuned.predict(X_test) )))
print(np.sqrt(mean_squared_error(y_test, np.expm1(gbm_tuned.predict(X_test)))),np.sqrt(mean_squared_log_error( y_test, np.expm1(gbm_tuned.predict(X_test)) )))

test1=scale(test)
son=np.expm1(gbm_tuned.predict(test1))
son=pd.DataFrame(son)
son.rename(columns={0:"SalePrice"},inplace=True)
son_sonuc=pd.concat([test2,son],axis=1)
son_sonuc.to_csv("submission.csv",index=False)